Model trained on contaxts being original_code. 

In [1]:
from transformers import (
    AutoTokenizer,
    TFAutoModelForQuestionAnswering,
    DataCollatorForSeq2Seq,
    keras_callbacks,
    TFAutoModelForSeq2SeqLM,
    TFEncoderDecoderModel,
)
import tensorflow as tf
from huggingface_hub import notebook_login
from question_answering.constants import constants
from question_answering.utils import core_qa_utils, generative_qa_utils
from question_answering.paths import generative_qa_paths

In [2]:
df_train, df_val, df_test = core_qa_utils.load_train_val_test_datasets(
    generative_qa_paths.python_dataset_dir
)

train_dataset, val_dataset, test_dataset = core_qa_utils.convert_dataframes_to_datasets(
    [df_train, df_val, df_test]
)

In [3]:
model_checkpoint = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
def tokenize_sample(sample, max_tokens=None, padding=False):
    question = sample["questions"].strip()
    context = sample["original_code"].strip()

    return tokenizer(question, context, max_length=max_tokens, padding=padding)

tokenized_train_dataset = train_dataset.map(tokenize_sample)
tokenized_val_dataset = val_dataset.map(tokenize_sample)
tokenized_test_dataset = test_dataset.map(tokenize_sample)

print(
    "Max number of tokens in tokenized train dataset: ",
    len(max(tokenized_train_dataset["input_ids"], key=len)),
)
print(
    "Max number of tokens in tokenized val dataset: ",
    len(max(tokenized_val_dataset["input_ids"], key=len)),
)
print(
    "Max number of tokens in tokenized test dataset: ",
    len(max(tokenized_test_dataset["input_ids"], key=len)),
)

Map:   0%|          | 0/56080 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1490 > 1024). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Max number of tokens in tokenized train dataset:  9244
Max number of tokens in tokenized val dataset:  575
Max number of tokens in tokenized test dataset:  901


In [5]:
max_length = 256


def filter_samples_below_number_of_tokens(dataset, max_tokens: int):
    indices_to_remove = []

    # Find indices of samples where number of tokens exceeds max number of tokens
    for index, sample in enumerate(dataset):
        tokenized_sample = tokenize_sample(sample)
        if len(tokenized_sample["input_ids"]) > max_tokens:
            indices_to_remove.append(index)

    # Keep only samples with number of tokens less or equal than max number of tokens
    dataset_indices = range(len(dataset))
    filtered_dataset = dataset.select(
        index for index in dataset_indices if index not in set(indices_to_remove)
    )

    return filtered_dataset


filtered_train_dataset = filter_samples_below_number_of_tokens(
    train_dataset, max_tokens=max_length
)
filtered_val_dataset = filter_samples_below_number_of_tokens(
    val_dataset, max_tokens=max_length
)
filtered_test_dataset = filter_samples_below_number_of_tokens(
    test_dataset, max_tokens=max_length
)

Parameter 'indices'=<generator object filter_samples_below_number_of_tokens.<locals>.<genexpr> at 0x0000015B827F9CB0> of the transform datasets.arrow_dataset.Dataset.select couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [6]:
print(
    "Number of samples in tokenized train dataset before filtering: ",
    len(train_dataset),
)
print("Number of samples in tokenized val dataset before filtering: ", len(val_dataset))
print(
    "Number of samples in tokenized test dataset before filtering: ", len(test_dataset)
)

print("\n---------------\n")

print(
    "Number of samples in tokenized train dataset after filtering: ",
    len(filtered_train_dataset),
)
print(
    "Number of samples in tokenized val dataset after filtering: ",
    len(filtered_val_dataset),
)
print(
    "Number of samples in tokenized test dataset after filtering: ",
    len(filtered_test_dataset),
)

Number of samples in tokenized train dataset before filtering:  56080
Number of samples in tokenized val dataset before filtering:  7000
Number of samples in tokenized test dataset before filtering:  7000

---------------

Number of samples in tokenized train dataset after filtering:  54930
Number of samples in tokenized val dataset after filtering:  6828
Number of samples in tokenized test dataset after filtering:  6854


In [7]:
def preprocess_dataset(dataset):
    questions = [q.strip() for q in dataset["questions"]]
    contexts = [c.strip() for c in dataset["original_code"]]
    answers = [c.strip() for c in dataset["answers"]]

    inputs = tokenizer(
        questions,
        contexts,
        # question_context,
        text_target=answers,
        max_length=max_length,
        padding="max_length",
        return_offsets_mapping=True,
    )

    return inputs

In [8]:
tokenized_train_dataset = filtered_train_dataset.map(
    preprocess_dataset,
    batched=True,
    remove_columns=filtered_train_dataset.column_names,
)
tokenized_val_dataset = filtered_val_dataset.map(
    preprocess_dataset,
    batched=True,
    remove_columns=filtered_val_dataset.column_names,
)
tokenized_test_dataset = filtered_test_dataset.map(
    preprocess_dataset,
    batched=True,
    remove_columns=filtered_test_dataset.column_names,
)

Map:   0%|          | 0/54930 [00:00<?, ? examples/s]

Map:   0%|          | 0/6828 [00:00<?, ? examples/s]

Map:   0%|          | 0/6854 [00:00<?, ? examples/s]

In [9]:
print(
    f"All tokenized train dataset entries have {max_length} tokens: ",
    all(
        [
            len(input_ids) == max_length
            for input_ids in tokenized_train_dataset["input_ids"]
        ]
    ),
)
print(
    f"All tokenized val dataset entries have {max_length} tokens: ",
    all(
        [
            len(input_ids) == max_length
            for input_ids in tokenized_val_dataset["input_ids"]
        ]
    ),
)
print(
    f"All tokenized test dataset entries have {max_length} tokens: ",
    all(
        [
            len(input_ids) == max_length
            for input_ids in tokenized_test_dataset["input_ids"]
        ]
    ),
)

All tokenized train dataset entries have 256 tokens:  True
All tokenized val dataset entries have 256 tokens:  True
All tokenized test dataset entries have 256 tokens:  True


In [10]:
# Model parameters
training_number = 5

model_name = "python-bart-uncased"
full_model_name = f"{model_name}-{training_number}"

# Checkpoints
checkpoint_filename_template = constants.checkpoint_filename_template
checkpoints_path = (
    generative_qa_paths.training_checkpoints_dir
    / full_model_name
    / checkpoint_filename_template
)

# Hub
hub_path = generative_qa_paths.hub_models_location / full_model_name

# Saved models
saved_models_path = generative_qa_paths.saved_models_dir / full_model_name

# Figures
figures_dir = generative_qa_paths.figures_dir / full_model_name

# Hyperparameters
batch_size = 8
train_epochs = 1

In [11]:
# Load model for fine-tuning
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBartForConditionalGeneration: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
- This IS expected if you are initializing TFBartForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBartForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBartForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


In [12]:
# Dataset preparation
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

tf_train_dataset = core_qa_utils.prepare_tf_dataset(
    model=model,
    hf_dataset=tokenized_train_dataset,
    collator=data_collator,
    batch_size=batch_size,
    shuffle=True
)

tf_val_dataset = core_qa_utils.prepare_tf_dataset(
    model=model,
    hf_dataset=tokenized_val_dataset,
    collator=data_collator,
    batch_size=batch_size,
)

tf_test_dataset = core_qa_utils.prepare_tf_dataset(
    model=model,
    hf_dataset=tokenized_test_dataset,
    collator=data_collator,
    batch_size=batch_size,
)

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [13]:
# Callbacks
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    checkpoints_path, verbose=1, save_weights_only=True
)
early_stop_cb = tf.keras.callbacks.EarlyStopping(patience=1)
push_to_hub = keras_callbacks.PushToHubCallback(
    output_dir=full_model_name, tokenizer=tokenizer
)

callbacks = [
    checkpoint_cb,
    early_stop_cb,
    # push_to_hub
]

Cloning https://huggingface.co/nlp-polish/python-bart-uncased-5 into local empty directory.


In [14]:
# Compile
num_train_steps = len(tf_train_dataset) * train_epochs

lr_scheduler = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_scheduler)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

# Compile
# loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# metrics = ["accuracy"]
# model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
# model.compile(optimizer=optimizer, metrics=metrics)
model.compile(optimizer=optimizer)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4070, compute capability 8.9


In [15]:
model.summary()

Model: "tf_bart_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (TFBartMainLayer)     multiple                  139420416 
                                                                 
 final_logits_bias (BiasLaye  multiple                 50265     
 r)                                                              
                                                                 
Total params: 139,470,681
Trainable params: 139,420,416
Non-trainable params: 50,265
_________________________________________________________________


In [16]:
# Fit the model on the new data
history = model.fit(
    tf_train_dataset,
    validation_data=tf_val_dataset,
    epochs=train_epochs,
    callbacks=callbacks,
)

6866/6866 [==============================] - ETA: 0s - loss: 2.6755
Epoch 1: saving model to e:\STUDIA\IPS\question-answering\generative-qa\training-checkpoints\python-bart-uncased-5\cp-01.ckpt
6866/6866 [==============================] - 1495s 215ms/step - loss: 2.6755 - val_loss: 2.7151


In [17]:
# Get best version of the model
best_model, best_epoch = core_qa_utils.get_best_model_from_checkpoints(
    model, history, model_name=full_model_name, remove_checkpoints=True, model_type="generative"
)

In [18]:
# Save best model's weights
generative_qa_utils.save_model(best_model, model_name=full_model_name)

In [19]:
# Load best model
loaded_model = generative_qa_utils.load_model(
    model_checkpoint, model_name=full_model_name
)
loaded_model.compile(optimizer=optimizer)

All PyTorch model weights were used when initializing TFBartForConditionalGeneration.

All the weights of TFBartForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


In [33]:
loaded_weights_model = generative_qa_utils.load_weights_into_model(
    model=model, 
    model_name=full_model_name
)

In [35]:
# Get predictions from the best model
loaded_model_evaluation = loaded_weights_model.evaluate(tf_test_dataset)

 20/857 [..............................] - ETA: 1:47 - loss: 1.9878

KeyboardInterrupt: 

In [22]:
import evaluate

metric = evaluate.load("bleu")
# rouge = evaluate.load("rouge")
# meteor = evaluate.load("meteor")
# bertscore = evaluate.load("bertscore")
# sacrebleu = evaluate.load("sacrebleu")

In [34]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm

def generate_with_xla(batch):
    return loaded_weights_model.generate(
        input_ids=batch["input_ids"],
        attention_mask=batch["attention_mask"],
        max_new_tokens=max_length,
    )


In [24]:
tokenized_test_dataset2 = tokenized_test_dataset.train_test_split(test_size=0.001)['test']
len(tokenized_test_dataset2[0]['input_ids'])

256

In [39]:
tokenizer.decode(tokenized_test_dataset[0]['input_ids'])

"<s>How does the code add a user in the given buckets default object access control list?</s></s>def add_bucket_default_owner bucket_name user_email storage_client storage Client bucket storage_client bucket bucket_name bucket acl reload bucket default_object_acl user user_email grant_owner bucket default_object_acl save print 'Addeduser{}asanownerinthedefaultaclonbucket{}'format user_email bucket_name</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

In [40]:
tokenizer

BartTokenizerFast(name_or_path='facebook/bart-base', vocab_size=50265, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True, special=True),
}

In [36]:
tf_test_dataset2 = core_qa_utils.prepare_tf_dataset(
    model=loaded_weights_model,
    hf_dataset=tokenized_test_dataset2,
    collator=data_collator,
    batch_size=batch_size,
)

In [37]:
import pandas as pd

all_preds = []
all_labels = []
all_labels2 = []
something = []
something2 = []
df = pd.DataFrame()

for batch, labels in tqdm(tf_test_dataset2):
    predictions = generate_with_xla(batch)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = labels.numpy()
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    all_preds.extend(decoded_preds)
    all_labels.extend(decoded_labels)
    all_labels2.extend(tokenizer.batch_decode(labels, skip_special_tokens=True))
    something.extend(tokenizer.batch_decode(batch['input_ids'], skip_special_tokens=True))
    something2.extend(tokenizer.batch_decode(batch['input_ids'], skip_special_tokens=True))
    data = {
        'qc': tokenizer.batch_decode(batch['input_ids'], skip_special_tokens=True),
        'labels': decoded_labels,
        'preds': decoded_preds
    }
    df = pd.DataFrame(data)

100%|██████████| 1/1 [00:04<00:00,  4.96s/it]


In [27]:
all_preds, all_labels, all_labels2, something, something2

(['a list of games',
  'all key combinations',
  'a string',
  'the text of an object',
  'the demo weather platform',
  'the rest controller',
  'a url'],
 [['a single game matching the given criteria'],
  ['the schema'],
  ['the current integer'],
  ['the names and default values of a functions arguments return list'],
  ['the demo weather'],
  ['filtering'],
  ['the current url for the given language code']],
 ['a single game matching the given criteria',
  'the schema',
  'the current integer',
  'the names and default values of a functions arguments return list',
  'the demo weather',
  'filtering',
  'the current url for the given language code'],
 ["What does one return?def one year week home away kind 'REG' started False infos _search_schedule year week home away kind started if not infos return Noneassert len infos 1 'Morethanonegamematchesthegivencriteria'return nflgame game Game infos[0]['eid']",
  'What does the code compare against the given data?def get_all_key_combinatio

In [28]:
df

,qc,labels,preds
0,What does one return?def one year week home aw...,[a single game matching the given criteria],a list of games
1,What does the code compare against the given d...,[the schema],all key combinations
2,What is representing the current page?def get_...,[the current integer],a string
3,What does the code get?def getargtxt obj one_a...,[the names and default values of a functions a...,the text of an object
4,What does the code setup?def setup_platform ha...,[the demo weather],the demo weather platform
5,What does s3siteautocompletewidget nt support ...,[filtering],the rest controller
6,What does the code translate?@register simple_...,[the current url for the given language code],a url


In [29]:
questions = []
contexts = []

for index, row in df.iterrows():
    questions.append(row['qc'].split('?')[0] + '?')
    contexts.append(row['qc'].split('?')[1])

df['questions'] = questions
df['contexts'] = contexts
labels = df['labels']
preds = df['preds']
df = df.drop(['qc', 'labels', 'preds'], axis=1)
df['labels'] = labels
df['preds'] = preds

In [30]:
df

,questions,contexts,labels,preds
0,What does one return?,def one year week home away kind 'REG' started...,[a single game matching the given criteria],a list of games
1,What does the code compare against the given d...,def get_all_key_combinations data flattened_sc...,[the schema],all key combinations
2,What is representing the current page?,def get_page self suffix try return int self R...,[the current integer],a string
3,What does the code get?,def getargtxt obj one_arg_per_line True args g...,[the names and default values of a functions a...,the text of an object
4,What does the code setup?,def setup_platform hass config add_devices dis...,[the demo weather],the demo weather platform
5,What does s3siteautocompletewidget nt support ...,def site def prep r if r representation 'json'...,[filtering],the rest controller
6,What does the code translate?,@register simple_tag takes_context True def tr...,[the current url for the given language code],a url


In [41]:
df.sample(n = 5)

,qc,labels,preds
5,What does s3siteautocompletewidget nt support ...,[filtering],the rest controller
3,What does the code get?def getargtxt obj one_a...,[the names and default values of a functions a...,the text of an object
4,What does the code setup?def setup_platform ha...,[the demo weather],the demo weather platform
1,What does the code compare against the given d...,[the schema],all key combinations
2,What is representing the current page?def get_...,[the current integer],a string


In [32]:
result = metric.compute(predictions=all_preds, references=all_labels)
result

{'bleu': 0.0,
 'precisions': [0.30434782608695654, 0.125, 0.1111111111111111, 0.0],
 'brevity_penalty': 0.5934874977560278,
 'length_ratio': 0.6571428571428571,
 'translation_length': 23,
 'reference_length': 35}